In [1]:
# Set up modules for Google functionality
from google.cloud import bigquery # To run BQ statements
from google_auth_oauthlib import flow # To authorise as user
from googleapiclient.discovery import build # To pull in from sheets, slides etc. API
from google.auth.transport.requests import Request

# Display
import pprint

# Operating system stuff
import pickle
import os.path
import sys

# Data handling
import json
import requests
from pandas import read_csv
import zipfile
import io
import numpy as np

# Stats, models, datasheets
import pandas as pd
import pyreadstat

# Visualisation
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib_venn # For venn diagrams
from pandas.plotting import autocorrelation_plot

# Network graphs
import networkx as nx

# Misc
from xlsxwriter.utility import xl_rowcol_to_cell # Used to create cell references
import itertools

# Load custom scripts in reusable_code folder
sys.path.append(r'/home/jupyter/reusable_code')

import google_api_functions as gaf
import my_qualtrics as qltx

In [2]:
# Create a qualtrics session
my_session=qltx.session('ExTDN3nBP2Q8vVVnUJeMsjUCbyuWPyEDxblfUqCW')

In [5]:
Leavers_survey_dict={
    'SV_00pthvlPe7DCx01':'Content'\
    ,'SV_1CgDK6uYdjqMv1H':'Expense'\
    ,'SV_eqyQ8hYm0MgFkbP':'Feature'\
    ,'SV_9EnSMGsy27qMqEd':'Other Service'}

#my_session.Get_list_of_surveys()

In [49]:
for n,key in enumerate(Leavers_survey_dict):
    print(key)
    print(n)
    survey_results=my_session.Download_survey_responses(key)
    survey_results['Survey Name']=Leavers_survey_dict[key]
    program = '''df_{0}=survey_results.copy()
df_{0}['Started']=pd.to_datetime(df_{0}['StartDate'])
df_{0}['Recorded']=pd.to_datetime(df_{0}['RecordedDate'])
df_{0}['week_start'] = (df_{0}['Recorded'] - df_{0}['Recorded'].dt.weekday.astype('timedelta64[D]')).dt.date
df_{0}['Status'] = pd.to_numeric(df_{0}['Status'])

    '''.format(Leavers_survey_dict[key].replace(' ','_'))
    exec(program)

    

In [65]:
df_Expense.groupby('week_start').agg({'Status':'count'}).rename(columns={'Status':'Expense'}).join(\
df_Feature.groupby('week_start').agg({'Status':'count'}).rename(columns={'Status':'Feature'}),how='inner').join(\
df_Content.groupby('week_start').agg({'Status':'count'}).rename(columns={'Status':'Content'}),how='inner').join(\
df_Other_Service.groupby('week_start').agg({'Status':'count'}).rename(columns={'Status':'Other Service'}),how='inner')

In [33]:
df_Feature.dtypes

In [41]:
df_Feature['week_start'] = (df_Feature['Started'] - df_Feature['Started'].dt.weekday.astype('timedelta64[D]')).dt.date
df_Feature['week_start'].unique()